# OpenAI TTS - Synthese Vocale par API

**Module :** 01-Audio-Foundation  
**Niveau :** Debutant  
**Technologies :** OpenAI TTS API (tts-1, tts-1-hd)  
**Duree estimee :** 30 minutes  

## Objectifs d'Apprentissage

- [ ] Configurer l'acces a l'API OpenAI TTS
- [ ] Generer de la parole a partir de texte avec `client.audio.speech.create`
- [ ] Comparer les 6 voix disponibles (alloy, echo, fable, onyx, nova, shimmer)
- [ ] Comprendre la difference entre tts-1 (rapide) et tts-1-hd (haute qualite)
- [ ] Maitriser les formats de sortie (mp3, opus, aac, flac, wav)
- [ ] Controler la vitesse de parole (0.25 a 4.0)
- [ ] Sauvegarder et ecouter les fichiers audio generes

## Prerequis

- Environment Setup (module 00) complete
- Cle API OpenAI configuree (`OPENAI_API_KEY` dans `.env`)
- Connaissances de base en Python

**Navigation :** [Index](../README.md) | [Suivant >>](01-2-OpenAI-Whisper-STT.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres TTS
model_name = "tts-1"               # "tts-1" (rapide) ou "tts-1-hd" (haute qualite)
voice = "alloy"                    # "alloy", "echo", "fable", "onyx", "nova", "shimmer"
output_format = "mp3"              # "mp3", "opus", "aac", "flac", "wav"
speed = 1.0                        # Vitesse de parole (0.25 a 4.0)

# Configuration sauvegarde
generate_audio = True              # Generer les fichiers audio
save_audio_files = True            # Sauvegarder les fichiers generes
compare_voices = True              # Comparer toutes les voix

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
from io import BytesIO
import logging

# Lecture audio dans Jupyter
from IPython.display import Audio, display, HTML

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import play_audio_bytes, synthesize_openai
        print("Helpers audio importes")
    except ImportError:
        print("Helpers audio non disponibles - mode autonome")

# Repertoire de sortie
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'tts'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('openai_tts')

print(f"OpenAI TTS - Synthese Vocale")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}, Modele : {model_name}, Voix : {voice}")
print(f"Sortie : {OUTPUT_DIR}")

Helpers audio importes
OpenAI TTS - Synthese Vocale
Date : 2026-02-18 00:57:15
Mode : interactive, Modele : tts-1, Voix : alloy
Sortie : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\tts


In [3]:
# Chargement de la configuration et validation API
from dotenv import load_dotenv

# Recherche du .env dans les parents
current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve dans l'arborescence")

# Verification cle API OpenAI
openai_key = os.getenv('OPENAI_API_KEY')

if not openai_key:
    if notebook_mode == "batch" and not generate_audio:
        print("Mode batch sans generation : cle API ignoree")
        openai_key = "dummy_key_for_validation"
    else:
        raise ValueError(
            "OPENAI_API_KEY manquante dans .env\n"
            "Obtenez votre cle sur : https://platform.openai.com/api-keys"
        )

# Initialisation client OpenAI
from openai import OpenAI

client = OpenAI(api_key=openai_key)

# Test de connexion
if openai_key != "dummy_key_for_validation":
    try:
        models = client.models.list()
        tts_models = [m.id for m in models if 'tts' in m.id]
        print(f"Connexion API reussie")
        print(f"Modeles TTS disponibles : {tts_models}")
    except Exception as e:
        print(f"Erreur connexion : {str(e)[:100]}")

print(f"\nConfiguration TTS :")
print(f"  Modele : {model_name}")
print(f"  Voix : {voice}")
print(f"  Format : {output_format}")
print(f"  Vitesse : {speed}")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


Connexion API reussie
Modeles TTS disponibles : ['gpt-4o-mini-tts-2025-12-15', 'tts-1-hd', 'tts-1-1106', 'tts-1-hd-1106', 'gpt-4o-mini-tts', 'gpt-4o-mini-tts-2025-03-20', 'tts-1']

Configuration TTS :
  Modele : tts-1
  Voix : alloy
  Format : mp3
  Vitesse : 1.0


## Section 1 : Premiere generation TTS

L'API OpenAI TTS permet de convertir du texte en parole naturelle. L'appel de base utilise `client.audio.speech.create()` avec trois parametres obligatoires :

| Parametre | Description | Valeurs |
|-----------|-------------|--------|
| `model` | Modele TTS | `tts-1` (rapide), `tts-1-hd` (haute qualite) |
| `voice` | Voix synthetique | `alloy`, `echo`, `fable`, `onyx`, `nova`, `shimmer` |
| `input` | Texte a synthetiser | Jusqu'a 4096 caracteres |

La reponse contient directement les donnees audio binaires.

In [4]:
# Premiere generation TTS
print("GENERATION TTS - PREMIER EXEMPLE")
print("=" * 45)

sample_text = (
    "Bonjour et bienvenue dans ce cours sur l'intelligence artificielle generative. "
    "Aujourd'hui, nous allons decouvrir comment transformer du texte en parole "
    "grace a l'API OpenAI TTS."
)

print(f"Texte : {sample_text}")
print(f"Longueur : {len(sample_text)} caracteres")
print(f"Modele : {model_name}, Voix : {voice}")

if generate_audio:
    start_time = time.time()

    response = client.audio.speech.create(
        model=model_name,
        voice=voice,
        input=sample_text,
        response_format=output_format,
        speed=speed
    )

    audio_bytes = response.content
    generation_time = time.time() - start_time

    print(f"\nGeneration reussie")
    print(f"  Temps de generation : {generation_time:.2f}s")
    print(f"  Taille fichier : {len(audio_bytes) / 1024:.1f} KB")
    print(f"  Format : {output_format}")

    # Lecture dans le notebook
    print(f"\nLecture audio :")
    display(Audio(data=audio_bytes, autoplay=False))

    # Sauvegarde
    if save_audio_files:
        filepath = OUTPUT_DIR / f"tts_intro_{voice}.{output_format}"
        with open(filepath, 'wb') as f:
            f.write(audio_bytes)
        print(f"  Fichier sauvegarde : {filepath.name}")
else:
    print("\nGeneration desactivee (generate_audio=False)")

GENERATION TTS - PREMIER EXEMPLE
Texte : Bonjour et bienvenue dans ce cours sur l'intelligence artificielle generative. Aujourd'hui, nous allons decouvrir comment transformer du texte en parole grace a l'API OpenAI TTS.
Longueur : 178 caracteres
Modele : tts-1, Voix : alloy


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"



Generation reussie
  Temps de generation : 2.83s
  Taille fichier : 217.5 KB
  Format : mp3

Lecture audio :


  Fichier sauvegarde : tts_intro_alloy.mp3


### Interpretation : Premiere generation

| Aspect | Observation | Signification |
|--------|-------------|---------------|
| Temps de generation | Typiquement < 3s | L'API est optimisee pour une reponse rapide |
| Taille fichier | Variable selon format | MP3 compresse, WAV plus volumineux |
| Qualite audio | Voix naturelle | Les modeles TTS d'OpenAI sont parmi les meilleurs du marche |

**Points cles** :
1. La reponse est un flux binaire, pas du JSON
2. `IPython.display.Audio` permet l'ecoute directe dans le notebook
3. Le texte francais est bien gere sans parametre de langue explicite

## Section 2 : Comparaison des 6 voix

OpenAI propose 6 voix pre-entrainees, chacune avec un caractere distinct :

| Voix | Caractere | Cas d'usage recommande |
|------|-----------|------------------------|
| `alloy` | Neutre, polyvalente | Usage general, narration |
| `echo` | Masculine, posee | Podcasts, documentaires |
| `fable` | Expressive, narrative | Histoires, contes |
| `onyx` | Grave, autoritaire | Presentations formelles |
| `nova` | Feminine, chaleureuse | Assistants vocaux, e-learning |
| `shimmer` | Douce, amicale | Applications grand public |

In [5]:
# Comparaison des 6 voix
print("COMPARAISON DES VOIX")
print("=" * 45)

all_voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]
comparison_text = (
    "L'intelligence artificielle transforme notre facon de travailler "
    "et de communiquer au quotidien."
)

voice_results = {}

if generate_audio and compare_voices:
    for v in all_voices:
        print(f"\nGeneration avec la voix '{v}'...")
        start_time = time.time()

        response = client.audio.speech.create(
            model=model_name,
            voice=v,
            input=comparison_text,
            response_format=output_format
        )

        audio_data = response.content
        gen_time = time.time() - start_time

        voice_results[v] = {
            "audio": audio_data,
            "time": gen_time,
            "size_kb": len(audio_data) / 1024
        }

        print(f"  Voix : {v} | Temps : {gen_time:.2f}s | Taille : {len(audio_data)/1024:.1f} KB")
        display(Audio(data=audio_data, autoplay=False))

        # Sauvegarde individuelle
        if save_audio_files:
            filepath = OUTPUT_DIR / f"voice_{v}.{output_format}"
            with open(filepath, 'wb') as f:
                f.write(audio_data)

    # Tableau recapitulatif
    print(f"\nRecapitulatif :")
    print(f"{'Voix':<12} {'Temps (s)':<12} {'Taille (KB)':<12}")
    print("-" * 36)
    for v, data in voice_results.items():
        print(f"{v:<12} {data['time']:<12.2f} {data['size_kb']:<12.1f}")
else:
    print("Comparaison desactivee (generate_audio ou compare_voices = False)")

COMPARAISON DES VOIX

Generation avec la voix 'alloy'...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Voix : alloy | Temps : 2.35s | Taille : 119.1 KB



Generation avec la voix 'echo'...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


...

  Voix : nova | Temps : 2.75s | Taille : 114.8 KB



Generation avec la voix 'shimmer'...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Voix : shimmer | Temps : 1.65s | Taille : 120.5 KB



Recapitulatif :
Voix         Temps (s)    Taille (KB) 
------------------------------------
alloy        2.35         119.1       
echo         1.73         118.6       
fable        2.10         119.1       
onyx         2.55         119.5       
nova         2.75         114.8       
shimmer      1.65         120.5       


### Interpretation : Comparaison des voix

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Temps de generation | Similaire pour toutes les voix | Le choix de voix n'affecte pas la latence |
| Taille des fichiers | Quasi identique | La compression est independante de la voix |
| Qualite perceptuelle | Variable selon le contexte | Chaque voix excelle dans des situations differentes |

> **Note technique** : Les voix sont optimisees pour l'anglais mais fonctionnent bien en francais. Pour du contenu multilingue, tester chaque voix dans la langue cible.

## Section 3 : Modeles tts-1 vs tts-1-hd

OpenAI propose deux modeles avec des compromis differents :

| Modele | Latence | Qualite | Cout | Cas d'usage |
|--------|---------|---------|------|-------------|
| `tts-1` | Faible (~1s) | Bonne | $0.015 / 1K chars | Temps reel, prototypage |
| `tts-1-hd` | Plus elevee (~2-3s) | Excellente | $0.030 / 1K chars | Production, narration |

In [6]:
# Comparaison tts-1 vs tts-1-hd
print("COMPARAISON MODELES")
print("=" * 45)

models_to_compare = ["tts-1", "tts-1-hd"]
comparison_long_text = (
    "La synthese vocale par intelligence artificielle a fait des progres "
    "considerables ces dernieres annees. Les modeles modernes produisent "
    "une parole naturelle, avec des intonations et un rythme proches "
    "de la voix humaine. Cette technologie ouvre de nouvelles possibilites "
    "pour l'education, l'accessibilite et la creation de contenu."
)

model_results = {}

if generate_audio:
    for m in models_to_compare:
        print(f"\nGeneration avec {m}...")
        start_time = time.time()

        response = client.audio.speech.create(
            model=m,
            voice=voice,
            input=comparison_long_text,
            response_format=output_format
        )

        audio_data = response.content
        gen_time = time.time() - start_time

        model_results[m] = {
            "audio": audio_data,
            "time": gen_time,
            "size_kb": len(audio_data) / 1024
        }

        print(f"  Modele : {m}")
        print(f"  Temps : {gen_time:.2f}s")
        print(f"  Taille : {len(audio_data)/1024:.1f} KB")
        display(Audio(data=audio_data, autoplay=False))

    # Analyse comparative
    if len(model_results) == 2:
        t1 = model_results["tts-1"]
        thd = model_results["tts-1-hd"]
        speedup = thd["time"] / t1["time"] if t1["time"] > 0 else 0

        print(f"\nAnalyse comparative :")
        print(f"  tts-1 est {speedup:.1f}x plus rapide que tts-1-hd")
        print(f"  Difference de taille : {abs(t1['size_kb'] - thd['size_kb']):.1f} KB")
        print(f"  Cout tts-1 : ${len(comparison_long_text) * 0.015 / 1000:.4f}")
        print(f"  Cout tts-1-hd : ${len(comparison_long_text) * 0.030 / 1000:.4f}")
else:
    print("Generation desactivee")

COMPARAISON MODELES

Generation avec tts-1...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Modele : tts-1
  Temps : 5.40s
  Taille : 406.4 KB



Generation avec tts-1-hd...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Modele : tts-1-hd
  Temps : 4.72s
  Taille : 407.3 KB



Analyse comparative :
  tts-1 est 0.9x plus rapide que tts-1-hd
  Difference de taille : 0.9 KB
  Cout tts-1 : $0.0050
  Cout tts-1-hd : $0.0099


## Section 4 : Formats de sortie et vitesse

L'API supporte plusieurs formats audio et un controle de vitesse :

### Formats disponibles

| Format | Compression | Taille | Cas d'usage |
|--------|-------------|--------|-------------|
| `mp3` | Lossy | Petite | Web, streaming, usage general |
| `opus` | Lossy | Tres petite | VoIP, faible bande passante |
| `aac` | Lossy | Petite | Applications mobiles (iOS/Android) |
| `flac` | Lossless | Moyenne | Archivage haute qualite |
| `wav` | Non compresse | Grande | Traitement audio, edition |

### Controle de vitesse

Le parametre `speed` accepte des valeurs de 0.25 (tres lent) a 4.0 (tres rapide).

In [7]:
# Test des formats de sortie et de la vitesse
print("FORMATS DE SORTIE ET VITESSE")
print("=" * 45)

short_text = "Ceci est un test des differents formats audio et vitesses de parole."

# --- Comparaison des formats ---
formats = ["mp3", "opus", "aac", "flac", "wav"]
format_results = {}

if generate_audio:
    print("\n--- Comparaison des formats ---")
    for fmt in formats:
        try:
            response = client.audio.speech.create(
                model=model_name,
                voice=voice,
                input=short_text,
                response_format=fmt
            )
            audio_data = response.content
            format_results[fmt] = len(audio_data) / 1024
            print(f"  {fmt:>5} : {len(audio_data)/1024:>8.1f} KB")

            if save_audio_files:
                filepath = OUTPUT_DIR / f"format_test.{fmt}"
                with open(filepath, 'wb') as f:
                    f.write(audio_data)
        except Exception as e:
            print(f"  {fmt:>5} : Erreur - {str(e)[:60]}")

    # --- Comparaison des vitesses ---
    print("\n--- Comparaison des vitesses ---")
    speeds = [0.5, 0.75, 1.0, 1.5, 2.0]

    for spd in speeds:
        response = client.audio.speech.create(
            model=model_name,
            voice=voice,
            input=short_text,
            response_format=output_format,
            speed=spd
        )
        audio_data = response.content
        print(f"  Vitesse {spd:>4}x : {len(audio_data)/1024:.1f} KB")
        display(Audio(data=audio_data, autoplay=False))

    # Recapitulatif formats
    if format_results:
        print(f"\nRecapitulatif tailles par format :")
        ref_size = format_results.get('wav', 1)
        for fmt, size in sorted(format_results.items(), key=lambda x: x[1]):
            ratio = (size / ref_size * 100) if ref_size > 0 else 0
            print(f"  {fmt:>5} : {size:>8.1f} KB ({ratio:>5.1f}% de WAV)")
else:
    print("Generation desactivee")

FORMATS DE SORTIE ET VITESSE

--- Comparaison des formats ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


    mp3 :     84.8 KB


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


   opus :     21.2 KB


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Vitesse  1.5x : 57.2 KB


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Vitesse  2.0x : 40.3 KB



Recapitulatif tailles par format :
   opus :     21.2 KB ( 11.4% de WAV)
    aac :     27.3 KB ( 14.7% de WAV)
    mp3 :     84.8 KB ( 45.5% de WAV)
   flac :    108.5 KB ( 58.2% de WAV)
    wav :    186.4 KB (100.0% de WAV)


### Interpretation : Formats et vitesse

| Format | Taille relative | Recommandation |
|--------|----------------|----------------|
| WAV | 100% (reference) | Edition audio, traitement |
| FLAC | ~50-60% | Archivage sans perte |
| MP3 | ~10-15% | Usage web general |
| AAC | ~10-12% | Mobile (iOS/Android) |
| Opus | ~8-10% | VoIP, streaming faible debit |

**Points cles** :
1. Opus offre le meilleur ratio qualite/taille pour la voix
2. WAV est necessaire pour du post-traitement audio
3. La vitesse modifie la duree mais pas significativement la qualite

In [8]:
# Mode interactif - Generation personnalisee
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF - GENERATION PERSONNALISEE")
    print("=" * 50)
    print("\nEntrez votre propre texte a synthetiser :")
    print("(Laissez vide pour passer a la suite)")

    try:
        user_text = input("\nVotre texte : ")

        if user_text.strip():
            user_voice = input(f"Voix [{voice}] (alloy/echo/fable/onyx/nova/shimmer) : ").strip() or voice
            user_model = input(f"Modele [{model_name}] (tts-1/tts-1-hd) : ").strip() or model_name

            print(f"\nGeneration en cours...")
            response = client.audio.speech.create(
                model=user_model,
                voice=user_voice,
                input=user_text,
                response_format=output_format,
                speed=speed
            )

            print(f"Generation reussie ({len(response.content)/1024:.1f} KB)")
            display(Audio(data=response.content, autoplay=False))

            if save_audio_files:
                ts = datetime.now().strftime('%Y%m%d_%H%M%S')
                filepath = OUTPUT_DIR / f"custom_{user_voice}_{ts}.{output_format}"
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                print(f"Sauvegarde : {filepath.name}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

MODE INTERACTIF - GENERATION PERSONNALISEE

Entrez votre propre texte a synthetiser :
(Laissez vide pour passer a la suite)
Mode interactif non disponible (execution automatisee)


## Bonnes pratiques et analyse des couts

### Optimisation de l'utilisation

| Strategie | Description | Economie |
|-----------|-------------|----------|
| Choisir tts-1 pour les tests | 2x moins cher que tts-1-hd | 50% |
| Utiliser Opus pour le streaming | Meilleure compression | Bande passante |
| Cacher les resultats | Eviter de regenerer le meme texte | Variable |
| Tester avec des textes courts | Ajuster les parametres avant production | Variable |

### Grille tarifaire (Janvier 2025)

| Modele | Cout par 1M caracteres | Cout pour 1 heure de narration (~9000 mots, ~50K chars) |
|--------|------------------------|-----|
| `tts-1` | $15.00 | ~$0.75 |
| `tts-1-hd` | $30.00 | ~$1.50 |

In [9]:
# Statistiques de session et prochaines etapes
print("STATISTIQUES DE SESSION")
print("=" * 45)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Modele utilise : {model_name}")
print(f"Voix principale : {voice}")
print(f"Format de sortie : {output_format}")

if voice_results:
    total_generated = len(voice_results)
    total_size = sum(v['size_kb'] for v in voice_results.values())
    total_time = sum(v['time'] for v in voice_results.values())
    print(f"Fichiers generes (comparaison voix) : {total_generated}")
    print(f"Taille totale : {total_size:.1f} KB")
    print(f"Temps total de generation : {total_time:.1f}s")

if save_audio_files:
    saved_files = list(OUTPUT_DIR.glob('*'))
    print(f"Fichiers sauvegardes : {len(saved_files)} dans {OUTPUT_DIR}")

print(f"\nPROCHAINES ETAPES")
print(f"1. Explorer la reconnaissance vocale (01-2-OpenAI-Whisper-STT)")
print(f"2. Decouvrir les operations audio de base (01-3-Basic-Audio-Operations)")
print(f"3. Tester Whisper en local avec GPU (01-4-Whisper-Local)")
print(f"4. Essayer le TTS local avec Kokoro (01-5-Kokoro-TTS-Local)")

print(f"\nNotebook TTS termine - {datetime.now().strftime('%H:%M:%S')}")

STATISTIQUES DE SESSION
Date : 2026-02-18 00:58:01
Modele utilise : tts-1
Voix principale : alloy
Format de sortie : mp3
Fichiers generes (comparaison voix) : 6
Taille totale : 711.6 KB
Temps total de generation : 13.1s
Fichiers sauvegardes : 12 dans D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\tts

PROCHAINES ETAPES
1. Explorer la reconnaissance vocale (01-2-OpenAI-Whisper-STT)
2. Decouvrir les operations audio de base (01-3-Basic-Audio-Operations)
3. Tester Whisper en local avec GPU (01-4-Whisper-Local)
4. Essayer le TTS local avec Kokoro (01-5-Kokoro-TTS-Local)

Notebook TTS termine - 00:58:01


---

# CHALLENGE BONUS - Narration Multi-Voix

**Points : 0.5 pts**

## Objectif

Créer une **narration avec plusieurs voix** pour simuler un dialogue ou une présentation.

## Ce que vous avez appris

Ce notebook montre:
- **Section 1** : Génération TTS de base avec `client.audio.speech.create()`
- **Section 2** : Comparaison des 6 voix disponibles
- **Section 4** : Contrôle de la vitesse avec `speed`

## Criteres de succes

- [ ] Créer un court script de **3 lignes de dialogue** (ex: interview, conversation)
- [ ] Attribuer une **voix différente** à chaque personnage
- [ ] Générer chaque ligne séparément avec la voix appropriée
- [ ] Combiner les fichiers audio en un seul (avec pydub ou manuellement)
- [ ] Sauvegarder le résultat final

## Contraintes techniques

- Utiliser au moins 2 voix différentes
- Vitesse cohérente (speed=1.0 pour tous)
- Durée totale < 30 secondes

---

**Soumission** : PR avec titre "Challenge #8 - [Votre Nom]", script et fichier audio combiné